In [5]:
import os
import sys

notebook_dir = os.getcwd() 

# Navigate up to the 'src' directory, then add 'project_ideate'
# Assuming your notebook is in 'notebook/' and 'src' is a sibling of 'notebook'
project_root = os.path.abspath(os.path.join(notebook_dir, '..')) # Goes from 'notebook' to project root
sys.path.append(os.path.join(project_root, 'src'))

# Agent Testing Notebook

This notebook provides a basic structure for testing the agents defined in:
* `src/project_ideate/agents/learning_agents.py`
* `src/project_ideate/agents/building_agents.py`
* `src/project_ideate/agents/organizing_agents.py`


## Import necessary libraries and agents

In [10]:
# Import agents from learning_agents.py
from project_ideate.agents.learning_agents import anthropologist, cross_pollinator, experimenter

# Import any other necessary libraries (e.g., for running agents, handling outputs)

## Define a test function or class

In [ ]:
import asyncio
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Assuming these imports work and define your agents correctly
from project_ideate.agents.learning_agents import *
from project_ideate.agents.building_agents import *
from project_ideate.agents.organizing_agents import *

# If not already imported from your files, ensure these are defined for the example:
# class MockSettings:
#     DEFAULT_MODEL = "gemini-1.5-flash-latest" # Replace with your actual model name
# settings = MockSettings()
# from google.adk.agents import Agent
# anthropologist = Agent(name="Anthropologist", model=settings.DEFAULT_MODEL, instruction="...", tools=[])
# # ... define all other agents similarly for a self-contained snippet ...


# --- Session Service (instantiate once outside the loop) ---
APP_NAME = "agent_testing_app"
USER_ID = "agent_user"
session_service = InMemorySessionService() # Instantiate session service only once

async def ask_agent_to_present_itself(agent):
    """Sends a prompt to the agent asking it to present itself using a unique session."""
    
    # Generate a UNIQUE session ID for each agent based on its name
    # This prevents cross-agent event warnings
    current_session_id = f"session_intro_{agent.name.lower().replace(' ', '_')}"
    
    # 1. CREATE THE SESSION FOR THIS SPECIFIC AGENT'S INTERACTION - MUST BE AWAITED!
    # This ensures a fresh, isolated session for each agent's turn.
    await session_service.create_session( # <-- This is now correctly inside the async function and awaited
        app_name=APP_NAME, user_id=USER_ID, session_id=current_session_id
    )

    # 2. Create the runner, linking it to the newly created unique session
    agent_runner = Runner(
        agent=agent,
        app_name=APP_NAME,
        session_service=session_service,
    )
    
    query_text = "Present yourself and your role based on your instructions."
    print(f"\n>>> Asking the {agent.name}: '{query_text}'")

    user_content = types.Content(role='user', parts=[types.Part(text=query_text)])

    final_response_content = "No response received."
    async for event in agent_runner.run_async(
        user_id=USER_ID, session_id=current_session_id, new_message=user_content
    ):
        if event.is_final_response() and event.content and event.content.parts:
            final_response_content = event.content.parts[0].text
            break

    print(f"<<< {agent.name}'s Response: {final_response_content}")
    print("-" * 50)

# --- Execute the async function directly in Jupyter ---
# Make sure all your persona agents (anthropologist, experimenter, etc.) are defined and available in this scope.
for agent_obj in [anthropologist, experimenter, director, collaborator, hurdler, experience_architect, set_designer, storyteller, caregiver, cross_pollinator]:
    await ask_agent_to_present_itself(agent_obj)